In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import math
import scipy as sp

def read_and_remove_columns(file_path, columns_to_remove=None):
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(file_path)

    # Check if columns to remove are provided
    if columns_to_remove:
        # Remove the specified columns
        df = df.drop(columns=columns_to_remove, errors='ignore')

    return df

pokemon_df = read_and_remove_columns('Pokemon.csv', ['#'])
pokemon_df.head(5)

Exception ignored in: <function _get_module_lock.<locals>.cb at 0x7949c75beef0>
Traceback (most recent call last):
  File "<frozen importlib._bootstrap>", line 204, in cb
KeyboardInterrupt: 


,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [2]:
pokemon_df = pokemon_df.where(pd.notnull(pokemon_df), None)
pokemon_df.head(5)

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,None,309,39,52,43,60,50,65,1,False


In [3]:
match_dct = {'Grass': {'x2': ['Water', 'Ground', 'Rock'], 'x1/2': ['Fire', 'Grass', 'Poison', 'Flying', 'Bug', 'Steel', 'Dragon'], 'x0': []},
             'Fire': {'x2': ['Grass', 'Ice', 'Bug', 'Steel'], 'x1/2': ['Water', 'Dragon', 'Rock', 'Fire'], 'x0': []},
             'Water': {'x2': ['Fire', 'Ground', 'Rock'], 'x1/2': ['Grass', 'Water', 'Dragon'], 'x0': []},
             'Bug': {'x2': ['Grass', 'Psychic', 'Dark'], 'x1/2': ['Fire', 'Fighting', 'Poison', 'Flying', 'Ghost', 'Steel', 'Fairy'], 'x0': []},
             'Normal': {'x2': [], 'x1/2': ['Rock', 'Steel'], 'x0': ['Ghost']},
             'Poison': {'x2': ['Grass', 'Fairy'], 'x1/2': ['Poison', 'Ground', 'Rock', 'Ghost'], 'x0': ['Steel']},
             'Electric': {'x2': ['Water', 'Flying'], 'x1/2': ['Dragon', 'Grass', 'Electric'], 'x0': ['Ground']},
             'Ground': {'x2': ['Water', 'Grass', 'Ice'], 'x1/2': ['Poison', 'Rock'], 'x0': ['Flying']},
             'Fairy': {'x2': ['Fighting', 'Dragon', 'Dark'], 'x1/2': ['Fire', 'Poison', 'Steel'], 'x0': []},
             'Fighting': {'x2': ['Normal', 'Ice', 'Rock', 'Dark', 'Steel'], 'x1/2': ['Poison', 'Flying', 'Psychic', 'Bug', 'Fairy'], 'x0': ['Ghost']},
             'Psychic': {'x2': ['Fighting', 'Poison'], 'x1/2': ['Psychic', 'Steel'], 'x0': ['Dark']},
             'Rock': {'x2': ['Fire', 'Ice', 'Flying', 'Bug'], 'x1/2': ['Fighting', 'Ground', 'Steel'], 'x0': []},
             'Ghost': {'x2': ['Ghost', 'Psychic'], 'x1/2': ['Dark'], 'x0': ['Normal']},
             'Ice': {'x2': ['Grass', 'Ground', 'Flying', 'Dragon'], 'x1/2': ['Fire', 'Water', 'Ice', 'Steel'], 'x0': []},
             'Dragon': {'x2': ['Dragon'], 'x1/2': ['Steel'], 'x0': ['Fairy']},
             'Dark': {'x2': ['Psychic', 'Ghost'], 'x1/2': ['Fighting', 'Dark', 'Fairy'], 'x0': []},
             'Steel': {'x2': ['Ice', 'Rock', 'Fairy'], 'x1/2': ['Fire', 'Water', 'Electric', 'Steel'], 'x0': []},
             'Flying': {'x2': ['Grass', 'Fighting', 'Bug'], 'x1/2': ['Electric', 'Rock', 'Steel'], 'x0': []}
             }

In [4]:
pokemon_df_short = pokemon_df
pokemon_df_short

,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Charmander,Fire,None,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
795,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [5]:
def battle(pokemon1, pokemon2, match_dct):
    tie = False
    def calculate_weights(type1_1, type1_2, type2_1, type2_2):
        def get_weight(attacking_type, defending_type):
            if defending_type in match_dct[attacking_type]['x2']:
                return 2
            elif defending_type in match_dct[attacking_type]['x1/2']:
                return 0.5
            elif defending_type in match_dct[attacking_type]['x0']:
                return 0
            else:
                return 1


        # Inizializza i pesi a 1 per ciascun tipo
        weight1 = 1
        weight2 = 1

        # Considera il primo Pokémon che attacca il secondo Pokémon
        weight1 *= get_weight(pokemon1['Type 1'], pokemon2['Type 1'])
        if pokemon2['Type 2']:  # Se il secondo Pokémon ha un secondo tipo
            weight1 *= get_weight(pokemon1['Type 1'], pokemon2['Type 2'])
        if pokemon1['Type 2']:  # Se il primo Pokémon ha un secondo tipo
            weight1 *= get_weight(pokemon1['Type 2'], pokemon2['Type 1'])
            if pokemon2['Type 2']:
                weight1 *= get_weight(pokemon1['Type 2'], pokemon2['Type 2'])

        # Considera il secondo Pokémon che attacca il primo Pokémon
        weight2 *= get_weight(pokemon2['Type 1'], pokemon1['Type 1'])
        if pokemon1['Type 2']:  # Se il primo Pokémon ha un secondo tipo
            weight2 *= get_weight(pokemon2['Type 1'], pokemon1['Type 2'])
        if pokemon2['Type 2']:  # Se il secondo Pokémon ha un secondo tipo
            weight2 *= get_weight(pokemon2['Type 2'], pokemon1['Type 1'])
            if pokemon1['Type 2']:
                weight2 *= get_weight(pokemon2['Type 2'], pokemon1['Type 2'])

        return weight1, weight2

    # Determina chi attacca per primo in base alla velocità
    if pokemon1['Speed'] >= pokemon2['Speed']:
        first, second = pokemon1, pokemon2
    else:
        first, second = pokemon2, pokemon1

    # Calcola i pesi dei danni (moltiplicatori di tipo) per entrambi i Pokémon
    weight1, weight2 = calculate_weights(first['Type 1'], first.get('Type 2'), second['Type 1'], second.get('Type 2'))

    # Ciclo di battaglia
    while True:
        # Primo Pokémon attacca il secondo
        if (first['Attack']/second['Defense'])>=(first['Sp. Atk']/second['Sp. Def']):
          damage = (((1760*(first['Attack'] / second['Defense']))/50)+2) * weight1
        else:
          damage = (((1760*(first['Sp. Atk'] / second['Sp. Def']))/50)+2) * weight1

        if (second['Attack']/first['Defense'])>=(second['Sp. Atk']/first['Sp. Def']):
          damage2 = (((1760*(second['Attack'] / first['Defense']))/50)+2) * weight2
        else:
          damage2 = (((1760*(second['Sp. Atk'] / first['Sp. Def']))/50)+2) * weight2

        percent=1
        if damage == 0 and damage2 != 0:
          return second, first, percent,tie
        elif damage != 0 and damage2 == 0:
          return first, second, percent,tie
        elif damage == 0 and damage2 == 0:
            if first['Total']>second['Total']:
                percent=round(1-(second['Total']/first['Total']),2)
                return first, second, percent,tie
            elif first['Total']<second['Total']:
                percent=round(1-(first['Total']/second['Total']),2)
                return second, first, percent,tie
            else:
                tie=True
                percent=0
                return first, second, percent,tie
        else:
          round1 = math.ceil(second['HP']/damage)
          round2 = math.ceil(first['HP']/damage2)
          if round1<round2:
              percent=round(1-(round1/round2), 2)
              return first, second, percent,tie
          elif round1>round2:
              percent=round(1-(round2/round1), 2)
              return second, first, percent,tie
          else:
              if pokemon1['Speed'] == pokemon2['Speed']:
                tie= True
              percent= 0.3
              return first, second, percent,tie


In [6]:
def create_graph_battle(df):
    # Initialize the graph
    G = nx.DiGraph()


    # Convert the dataframe to a list of dictionaries once to avoid using df.loc repeatedly
    pokemon_list = df.to_dict('records')

    # Precompute all possible combinations of rows (pokemon pairs)
    for comb in itertools.combinations(pokemon_list, 2):
        pokemon1 = comb[0].copy()
        pokemon2 = comb[1].copy()

        # Simulate battle and add the result to the graph
        winner, loser, percent,tie = battle(pokemon1, pokemon2, match_dct)

        G.add_edge(winner['Name'], loser['Name'], weight=percent)
        if tie:
         G.add_edge(loser['Name'], winner['Name'], weight=0.3)

    return G

In [7]:
# Helper function to sort metrics and print top 3 nodes
def print_top_n(metric_dict, metric_name, n=3, pg=False):
    sorted_metric = sorted(metric_dict.items(), key=lambda x: x[1], reverse=True)[:n]
    print(f"\n\033[1;32mTop {n} nodes by {metric_name}:\033[0m")
    for node, score in sorted_metric:
        if pg:
          print(f"{node}: {score:.8f}")
        else:
          print(f"{node}: {score:.2f}")

def print_last_n(metric_dict, metric_name, n=3, pg=False):
    sorted_metric = sorted(metric_dict.items(), key=lambda x: x[1], reverse=True)[-n:]
    print(f"\n\033[1;32mLast {n} nodes by {metric_name}:\033[0m")
    for node, score in sorted_metric:
        if pg:
          print(f"{node}: {score:.8f}")
        else:
          print(f"{node}: {score:.2f}")

In [8]:
Graph = create_graph_battle(pokemon_df_short)

In [9]:
def print_results(Graph):
  # 1. In-degree and Out-degree for each node
  in_degrees_weight = dict(Graph.in_degree(weight='weight'))
  out_degrees_weight = dict(Graph.out_degree(weight='weight'))
  in_degrees = dict(Graph.in_degree())
  out_degrees = dict(Graph.out_degree())

  # Print top 10 nodes by in-degree and out-degree
  print_top_n(in_degrees, "In-Degree")
  print_top_n(out_degrees, "Out-Degree")

  print_top_n(in_degrees_weight, "In-Degree")
  print_top_n(out_degrees_weight, "Out-Degree")

  # 2. PageRank
  pagerank = nx.pagerank(Graph)
  print_top_n(pagerank, "PageRank", pg=True)
  print_last_n(pagerank, "PageRank", pg=True)

  # Compute HITS (Hub and Authority)
  hits_hub, hits_authority = nx.hits(Graph)

  print_top_n(hits_hub, "Hub Scores", pg=True)
  print_top_n(hits_authority, "Authority Scores", pg=True)

  # Global reciprocity
  global_reciprocity = nx.reciprocity(Graph)
  print("\nGlobal Reciprocity:", global_reciprocity)

In [10]:
print_results(Graph)


Top 3 nodes by In-Degree:
Happiny: 798.00
Magikarp: 795.00
Feebas: 795.00

Top 3 nodes by Out-Degree:
Arceus: 789.00
Regigigas: 782.00
Slaking: 773.00

Top 3 nodes by In-Degree:
Feebas: 660.67
Magikarp: 657.28
Happiny: 651.55

Top 3 nodes by Out-Degree:
Arceus: 518.50
KyuremBlack Kyurem: 514.23
Regigigas: 511.22

Top 3 nodes by PageRank:
Magikarp: 0.01695978
Feebas: 0.01002002
Happiny: 0.00966550

Last 3 nodes by PageRank:
Slaking: 0.00019938
Regigigas: 0.00019486
Arceus: 0.00019134

Top 3 nodes by Hub Scores:
Arceus: 0.00250000
Regigigas: 0.00245699
KyuremBlack Kyurem: 0.00243301

Top 3 nodes by Authority Scores:
Feebas: 0.00302434
Magikarp: 0.00300970
Happiny: 0.00299325

Global Reciprocity: 0.009082471079991777


In [11]:
def find_directed_cliques(Graph):
    # Step 1: Filter edges to keep only reciprocal edges
    reciprocal_edges = [(u, v) for u, v in Graph.edges() if Graph.has_edge(v, u)]
    reciprocal_Graph = nx.DiGraph()
    reciprocal_Graph.add_edges_from(reciprocal_edges)

    # Step 2: Find all cliques in the undirected version of the reciprocal graph
    undirected_reciprocal_Graph = reciprocal_Graph.to_undirected()
    undirected_cliques = list(nx.find_cliques(undirected_reciprocal_Graph))

    # Step 3: Filter cliques to ensure they are directed cliques in the original graph
    directed_cliques = []
    for clique in undirected_cliques:
        if len(clique) < 3:  # Skip cliques smaller than size 3
            continue

        # Check if the subgraph induced by the clique is strongly connected
        subgraph = Graph.subgraph(clique)
        if nx.is_strongly_connected(subgraph):
            directed_cliques.append(clique)

    return directed_cliques
directed_clique = find_directed_cliques(Graph)
print("directed clique in the network:", directed_clique)
length = len(directed_clique)
print(length)

directed clique in the network: [['Tynamo', 'Rufflet', 'Vullaby'], ['Tynamo', 'Horsea', 'Fennekin', 'Riolu'], ['Tynamo', 'Horsea', 'Mime Jr.'], ['Tynamo', 'Horsea', 'Meditite'], ['Charizard', 'Typhlosion', 'Fearow'], ['Charizard', 'Typhlosion', 'Staraptor'], ['Ducklett', 'Eevee', 'Blissey'], ['Ducklett', 'Eevee', 'Psyduck'], ['Ducklett', 'Kabuto', 'Blissey'], ['Ducklett', 'Noibat', 'Psyduck'], ['Sableye', 'Marshtomp', 'Lombre'], ['Sableye', 'Marshtomp', 'Prinplup'], ['Swampert', 'Abomasnow', 'Ivysaur'], ['Graveler', 'Omanyte', 'Tyrogue'], ['Graveler', 'Cacnea', 'Shroomish'], ['Graveler', 'Cacnea', 'Cherubi'], ['Roggenrola', 'Cleffa', 'Wooper'], ['Mightyena', 'Cloyster', 'Mankey'], ['Mightyena', 'Cloyster', 'Butterfree'], ['Mightyena', 'Cloyster', 'Noctowl'], ['Mightyena', 'Luxray', 'Ludicolo'], ['Mightyena', 'Victreebel', 'Mantine'], ['Mightyena', 'Castform', 'Mankey'], ['Mightyena', 'Castform', 'Mantine'], ['Mightyena', 'Castform', 'Dragonair', 'Noctowl'], ['Mightyena', 'Castform', 'B

In [13]:
from collections import Counter
clique_sizes = [len(clique) for clique in directed_clique]
size_distribution = Counter(clique_sizes)
size_distribution

Counter({3: 310, 4: 83, 5: 17})

In [14]:
node_counts = Counter([node for clique in directed_clique for node in clique])
node_counts.most_common(10)

[('Zorua', 20),
 ('Houndour', 16),
 ('Cloyster', 14),
 ('Horsea', 13),
 ('Sandslash', 13),
 ('Spinda', 13),
 ('Carvanha', 13),
 ('Electrike', 13),
 ('Marshtomp', 12),
 ('Cacnea', 11)]

**DIVISIONE PER TIPI DI POKEMON**

In [15]:
pokemon_df_short = pokemon_df

In [16]:
bw_per_type = {}

for ptype in pokemon_df_short['Type 1'].unique():
  print(f"\033[1;33m{ptype.upper()}\033[0m")

  pokemon_df_type = pokemon_df_short[pokemon_df_short['Type 1'] == ptype]
  Graph_type = create_graph_battle(pokemon_df_type)

  print_results(Graph_type)

  out_degrees_type = dict(Graph_type.out_degree())
  max_out_degree = sorted(out_degrees_type.items(), key=lambda x: x[1], reverse=True)[:1]
  in_degrees_type = dict(Graph_type.in_degree())
  max_in_degree = sorted(in_degrees_type.items(), key=lambda x: x[1], reverse=True)[:1]
  bw_per_type[ptype] = max_out_degree
  bw_per_type[ptype].append(max_in_degree)

  print(max_out_degree)
  print(max_in_degree)

  print()

GRASS

Top 3 nodes by In-Degree:
Sunkern: 69.00
Seedot: 67.00
Cherubi: 64.00

Top 3 nodes by Out-Degree:
ShayminLand Forme: 65.00
Ferrothorn: 64.00
Gogoat: 60.00

Top 3 nodes by In-Degree:
Sunkern: 57.08
Seedot: 50.85
Hoppip: 46.80

Top 3 nodes by Out-Degree:
Ferrothorn: 48.89
Amoonguss: 43.26
ShayminLand Forme: 43.09

Top 3 nodes by PageRank:
Sunkern: 0.09099193
Budew: 0.06394401
Seedot: 0.05795406

Last 3 nodes by PageRank:
Ferrothorn: 0.00383914
AbomasnowMega Abomasnow: 0.00372011
ShayminLand Forme: 0.00342578

Top 3 nodes by Hub Scores:
Ferrothorn: 0.02812027
VenusaurMega Venusaur: 0.02648606
ShayminLand Forme: 0.02626204

Top 3 nodes by Authority Scores:
Sunkern: 0.03292216
Seedot: 0.03059882
Hoppip: 0.02957188

Global Reciprocity: 0.007425742574257425
[('ShayminLand Forme', 65), [('Sunkern', 69)]]
[('Sunkern', 69)]

FIRE

Top 3 nodes by In-Degree:
Vulpix: 50.00
Slugma: 50.00
Torchic: 48.00

Top 3 nodes by Out-Degree:
Entei: 48.00
DarmanitanZen Mode: 48.00
Ho-oh: 46.00

Top 3 node

In [17]:
print(bw_per_type)

{'Grass': [('ShayminLand Forme', 65), [('Sunkern', 69)]], 'Fire': [('Entei', 48), [('Vulpix', 50)]], 'Water': [('KyogrePrimal Kyogre', 110), [('Magikarp', 111)]], 'Bug': [('PinsirMega Pinsir', 63), [('Caterpie', 62)]], 'Normal': [('Arceus', 96), [('Happiny', 97)]], 'Poison': [('Muk', 26), [('Ekans', 25)]], 'Electric': [('Raikou', 41), [('Pichu', 43)]], 'Ground': [('Groudon', 30), [('LandorusIncarnate Forme', 28)]], 'Fairy': [('Xerneas', 16), [('Cleffa', 16)]], 'Fighting': [('LucarioMega Lucario', 23), [('Tyrogue', 25)]], 'Psychic': [('MewtwoMega Mewtwo Y', 54), [('Ralts', 55)]], 'Rock': [('TyranitarMega Tyranitar', 40), [('Roggenrola', 40)]], 'Ghost': [('GiratinaAltered Forme', 31), [('Litwick', 31)]], 'Ice': [('Walrein', 21), [('Delibird', 22)]], 'Dragon': [('RayquazaMega Rayquaza', 29), [('Goomy', 31)]], 'Dark': [('Hydreigon', 27), [('Poochyena', 30)]], 'Steel': [('Dialga', 25), [('Beldum', 22)]], 'Flying': [('Noivern', 3), [('Noibat', 3)]]}


**DIVISIONE PER GENERAZIONE**

In [18]:
pokemon_df_short = pokemon_df

In [19]:
# Divisione dei pokemon per generazione

bw_per_gen = {}

for pgeneration in pokemon_df_short['Generation'].unique():
  print(pgeneration)
  pokemon_df_gen = pokemon_df_short[pokemon_df_short['Generation'] == pgeneration]
  Graph_gen = create_graph_battle(pokemon_df_gen)

  in_degrees_gen = dict(Graph_gen.in_degree())
  out_degrees_gen = dict(Graph_gen.out_degree())

  print_top_n(in_degrees_gen, "In-Degree")
  print_top_n(out_degrees_gen, "Out-Degree")

  max_out_degree = sorted(out_degrees_gen.items(), key=lambda x: x[1], reverse=True)[:1]
  max_in_degree = sorted(in_degrees_gen.items(), key=lambda x: x[1], reverse=True)[:1]
  bw_per_gen[pgeneration] = max_out_degree
  bw_per_gen[pgeneration].append(max_in_degree)

  print(max_out_degree)
  print(max_in_degree)

  print()

1

Top 3 nodes by In-Degree:
Magikarp: 165.00
Caterpie: 155.00
Metapod: 154.00

Top 3 nodes by Out-Degree:
Mewtwo: 160.00
MewtwoMega Mewtwo X: 159.00
KangaskhanMega Kangaskhan: 157.00
[('Mewtwo', 160), [('Magikarp', 165)]]
[('Magikarp', 165)]

2

Top 3 nodes by In-Degree:
Igglybuff: 102.00
Sentret: 101.00
Cleffa: 101.00

Top 3 nodes by Out-Degree:
Suicune: 94.00
Feraligatr: 89.00
TyranitarMega Tyranitar: 89.00
[('Suicune', 94), [('Igglybuff', 102)]]
[('Igglybuff', 102)]

3

Top 3 nodes by In-Degree:
Feebas: 158.00
Poochyena: 150.00
Zigzagoon: 148.00

Top 3 nodes by Out-Degree:
Slaking: 149.00
KyogrePrimal Kyogre: 148.00
Groudon: 146.00
[('Slaking', 149), [('Feebas', 158)]]
[('Feebas', 158)]

4

Top 3 nodes by In-Degree:
Happiny: 120.00
Kricketot: 117.00
Burmy: 114.00

Top 3 nodes by Out-Degree:
Arceus: 119.00
Regigigas: 113.00
GarchompMega Garchomp: 108.00
[('Arceus', 119), [('Happiny', 120)]]
[('Happiny', 120)]

5

Top 3 nodes by In-Degree:
Patrat: 162.00
Lillipup: 156.00
Tynamo: 149.

In [20]:
print(bw_per_gen)

{1: [('Mewtwo', 160), [('Magikarp', 165)]], 2: [('Suicune', 94), [('Igglybuff', 102)]], 3: [('Slaking', 149), [('Feebas', 158)]], 4: [('Arceus', 119), [('Happiny', 120)]], 5: [('KyuremBlack Kyurem', 157), [('Patrat', 162)]], 6: [('Zygarde50% Forme', 70), [('Scatterbug', 80)]]}


**ELIMINATI POKEMON LEGGENDARI E ALTERED FORMS**

In [21]:
pokemon_wo_legendary_df = pokemon_df[pokemon_df['Legendary'] == False]
pokemon_df_short_wo_legendary = pokemon_wo_legendary_df

In [22]:
keywords = [
    "Mega", "♀", "♂", "Primal", "Sandy", "Trash", "Wash", "Frost",
    "Fan", "Mow", "Female", "Unbound",
    "DeoxysAttack Forme", "DeoxysDefense Forme", "DeoxysSpeed Forme",
    "GiratinaAltered Forme", "ShayminSky Forme", "DarmanitanZen Mode",
    "TornadusTherian Forme", "ThundurusIncarnate Forme", "LandorusTherian Forme",
    "KyuremBlack Kyurem", "KyuremWhite Kyurem", "ThundurusTherian Forme",
    "KeldeoResolute Forme", "MeloettaPirouette Forme", "AegislashShield Forme",
    "PumpkabooSmall Size", "PumpkabooLarge Size", "PumpkabooSuper Size",
    "GourgeistSmall Size", "GourgeistLarge Size", "GourgeistSuper Size", "Gourgeist"
]

# Filtra il dataset per rimuovere i Pokémon con nomi contenenti una delle parole chiave
pokemon_df_nol_sv = pokemon_df_short_wo_legendary[~pokemon_df['Name'].str.contains('|'.join(keywords), case=False, na=False)]

<ipython-input-22-77c7760ac119>:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  pokemon_df_nol_sv = pokemon_df_short_wo_legendary[~pokemon_df['Name'].str.contains('|'.join(keywords), case=False, na=False)]


In [23]:
Graph_wol_sv = create_graph_battle(pokemon_df_nol_sv)

In [24]:
print_results(Graph_wol_sv)


Top 3 nodes by In-Degree:
Happiny: 669.00
Magikarp: 666.00
Feebas: 666.00

Top 3 nodes by Out-Degree:
Slaking: 664.00
Snorlax: 636.00
Cresselia: 617.00

Top 3 nodes by In-Degree:
Feebas: 545.33
Magikarp: 542.15
Happiny: 539.97

Top 3 nodes by Out-Degree:
Slaking: 445.00
Snorlax: 405.28
Cresselia: 399.34

Top 3 nodes by PageRank:
Magikarp: 0.01903854
Feebas: 0.01124567
Happiny: 0.01087696

Last 3 nodes by PageRank:
Manaphy: 0.00027595
Snorlax: 0.00024827
Slaking: 0.00022781

Top 3 nodes by Hub Scores:
Slaking: 0.00305190
Snorlax: 0.00288671
Cresselia: 0.00283887

Top 3 nodes by Authority Scores:
Feebas: 0.00366317
Magikarp: 0.00364032
Happiny: 0.00363809

Global Reciprocity: 0.010672991402312482
